In [3]:
%load_ext autoreload
%autoreload 2

import helper
import numpy as np

In [4]:
# read in training data
train_data_path = '\\\\SEAGATE-D4/Documents/My Hoang Nguyen/ML-SDrive/Sentiment Analysis/data/labeledTrainData.tsv/labeledTrainData.tsv'
train = helper.read_in_data(train_data_path)

print(train.shape)
train.head()

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [12]:
train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [5]:
# read in test data
test_data_path = '\\\\SEAGATE-D4/Documents/My Hoang Nguyen/ML-SDrive/Sentiment Analysis/data/testData.tsv/testData.tsv'
test = helper.read_in_data(test_data_path)

print(test.shape)
test.head()

(25000, 2)


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [6]:
# load Google's pre-trained word2vec
pretrained_word2vec_path = '\\\\SEAGATE-D4/Documents/My Hoang Nguyen/ML-SDrive/Sentiment Analysis/data/GoogleNews-vectors-negative300.bin'
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format(pretrained_word2vec_path, binary=True)        

c:\users\my hoang nguyen\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
# for k in word2vec.vocab.keys():
#     print(k)
#     print(word2vec.vocab[k])
# #     break

In [7]:
print(len(word2vec['around_pig_Ramfjord']))
word2vec['around_pig_Ramfjord'][:10]

300


array([ 0.265625  ,  0.02001953, -0.01037598,  0.10449219, -0.03613281,
        0.03540039,  0.09228516, -0.03295898, -0.03588867,  0.06738281], dtype=float32)

In [10]:
# format texts into tensor to feed into CNN
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

text_train = train['review'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)
sequences = tokenizer.texts_to_sequences(text_train) # list, same len as text_train. represent word as rank/index

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data_train = pad_sequences(sequences)
'''
data_train has shape of (n_reviews, word_index). 
Each review contains multiple sentences, each word in a sentence is formatted to an integer representing 
the word_index of the tokenizer
'''
print('data_train.shape', data_train.shape)

Found 88582 unique tokens.
data_train.shape (25000, 2493)


In [36]:
print('text_train[0] \n', text_train[0])
print('len(text_train[0]) \n', len(text_train[0]))
print('len(sequences[0]) \n', len(sequences[0]))
print('sequences[0] \n', sequences[0])

text_train[0] 
 With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when

In [39]:
tokenizer.word_index['latter']

1559

In [16]:
data_train[0][-10:]

array([   88, 19501, 19502,    10,   437,    26,     6,    21,     1,  1559])

In [40]:
# create train & validation data
from keras.utils import to_categorical
labels = to_categorical(np.asarray(train['sentiment']))

# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.2
indices = np.arange(data_train.shape[0])
np.random.shuffle(indices)
data_train = data_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data_train.shape[0])

x_train = data_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)
print('x_val.shape', x_val.shape)
print('y_val.shape', y_val.shape)

x_train.shape (20000, 2493)
y_train.shape (20000, 2)
x_val.shape (5000, 2493)
y_val.shape (5000, 2)


In [41]:
# create embedding_matrix
vocab_size = len(word_index)
EMBEDDING_DIM = 300 # this is from the pretrained vectors

embedding_matrix = np.zeros((vocab_size + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec:
        embedding_vector = word2vec[word]
    else:
        embedding_vector = None
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [42]:
data_train.shape

(25000, 2493)

In [58]:
# create embedding layer
from keras.layers import Embedding

input_length = data_train.shape[1] # len (num words) of longest review
embedding_layer = Embedding(vocab_size + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=input_length
                            , trainable=False) 

In [50]:
# a small 1D convnet
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model

n_labels = 2
sequence_input = Input(shape=(input_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(n_labels, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 33s - loss: 0.6942 - acc: 0.4986 - val_loss: 0.6935 - val_acc: 0.4968
Epoch 2/2
20000/20000 [==============================] - 29s - loss: 0.6935 - acc: 0.4964 - val_loss: 0.6933 - val_acc: 0.4968


In [51]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4189942563794239504
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 199016448
locality {
  bus_id: 1
}
incarnation: 12971655689657738912
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0"
]


In [57]:
import tensorflow as tf

# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)

# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


# Kim CNN

###### - Apply CNN non-static, initialized with word2vec

In [61]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from keras.models import Model

In [64]:
# parameters
n_labels = 2
filter_sizes = [2,3,4]
n_filter = 3

# input
sequence_input = Input(shape=(input_length,), dtype='int32')
# embedding: vector representation of word
embedded_sequences = embedding_layer(sequence_input)
# conv layer
features = []
for filter_size in filter_sizes:
    # conv layer
    x = Conv1D(n_filter, filter_size, activation='relu')(embedded_sequences)
    # global max pooling
    x = GlobalMaxPooling1D()(x)
    # add features together
    features.append(x)
# penultimate layer
z = Concatenate()(features)
# dropout
z = Dropout(0.5)(z)
# fully connected layer
z = Dense(128, activation='relu')(z)
preds = Dense(n_labels, activation='softmax')(z)

model = Model(sequence_input, preds)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 2493)          0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 2493, 300)     26574900    input_4[0][0]                    
____________________________________________________________________________________________________
conv1d_7 (Conv1D)                (None, 2492, 3)       1803        embedding_2[3][0]                
____________________________________________________________________________________________________
conv1d_8 (Conv1D)                (None, 2491, 3)       2703        embedding_2[3][0]                
___________________________________________________________________________________________